# Femlalogy SOC Report

**Project Title:** Threat Detection & Incident Response — *Using Wireshark, pfSense, and Wazuh*  
**Organization:** Femlalogy  
**Role:** Security Operations Center (SOC) Analyst


<img src="pic/fem1.png" alt="Femlalogy SOC dashboard" />


## 1. Executive Summary

The purpose of this project was to **strengthen Femlalogy’s cybersecurity posture** by deploying and integrating three key tools: **Wazuh**, **Wireshark**, and **pfSense**.  
In this project, we **simulated real-world attacks**, **detected network anomalies**, **enforced firewall rules**, and **responded to security incidents** effectively and promptly.

---

### 🔧 Tools & Their Roles

- **Wireshark:** Captured and analyzed suspicious traffic such as **Nmap scans** and **ping sweeps**.  
- **pfSense (with Snort):** Configured and enforced **network/firewall policies**, blocking **ransomware-related IPs** and **malicious ICMP traffic**.  
- **Wazuh:** Provided **security event correlation and monitoring**, generating **actionable alerts** and **incident reports**.

---

### 🧩 Key Takeaway

This project demonstrates the **importance of proactive monitoring** and a **layered defense strategy** in protecting organizational assets from evolving cyber threats.


## 2. Project Introduction

Cybersecurity is a cornerstone of success in today’s digital landscape.  
This project focused on **building and assessing a Security Operations Center (SOC)** for **Femlalogy**, designed to **detect, monitor, and respond** to cyber threats in **real time**.

---

### 🎯 Project Overview

- **Objective:** Establish a functional SOC that integrates monitoring, detection, and incident response capabilities.  
- **Scope:** Evaluate how effectively Femlalogy’s network can defend itself under simulated cyberattacks.  

---

### 🧰 Tools Used

| Tool | Purpose |
|------|----------|
| **Wazuh** | Event correlation, log monitoring, and alert generation. |
| **pfSense (with Snort)** | Network barrier and firewall enforcement against malicious traffic. |
| **Wireshark** | Deep packet inspection and analysis of suspicious network activity. |

---

### 🧪 Simulation Goal

Test Femlalogy’s **readiness against real-world cyber threats** by simulating reconnaissance, denial-of-service, and intrusion attempts.

---

### 👩🏾‍💻 Roles Summary

- **Wireshark:** Used for **packet capture and traffic analysis**.  
- **Wazuh:** Used for **event correlation, intrusion detection, and reporting**.  
- **pfSense:** Used as the **first line of defense**, enforcing firewall and intrusion prevention policies.


## 3. Methodology

This section outlines the practical steps and roles each tool played during the exercise.

---

### Tools & Actions

- **Wireshark**
  - Captured suspicious network traffic for deep packet inspection.
  - Focused on behaviors such as **Nmap SYN scans** and **ping sweeps (ICMP)**.

- **pfSense & Snort**
  - Enforced perimeter security and intrusion detection.
  - Blocked malicious **ICMP** packets and logged intrusion events via Snort rules.

- **Wazuh**
  - Collected and correlated host and network logs.
  - Generated alerts, tracked events, and produced executive-style incident reports.

---

### Lab Setup (topology)

- **Attacker:** Kali Linux
- **Victim:** Ubuntu   
- **Perimeter:** pfSense firewall with Snort — logging and blocking layer

Simple logical diagram:

[Kali (Attacker)] <---> [pfSense + Snort] <---> [Ubuntu (Victim + Wazuh agent)]

# Phase-by-Phase Analysis

## Phase 1 — Wireshark: Network Traffic Capture & Analysis

**Goal:** Capture and analyze suspicious traffic such as Nmap scans and ping sweeps.

---

### 1) Launch Wireshark (Victim machine)
1. Open **Wireshark** from your applications menu.
2. Select the correct network interface (common names: `eth0`, `ens33`, `enp0s3`).
   - Choose the interface with active traffic (packet counter increasing).
3. Click **Start Capture** (blue shark fin icon).

> Tip: Think of this as turning on a surveillance camera for your network — leave it running while you generate traffic.

---

### 2) Apply Filters to Focus Your View
Wireshark captures everything. Use display filters to zoom in on relevant traffic.

**Common display filters (replace `192.168.1.xxx` with the victim IP):**

ip.addr == 192.168.1.xxx # show traffic to/from victim
icmp # show ICMP (ping) packets
dns # show DNS queries/responses
http # show web browsing traffic
tcp.port == 22 # show SSH traffic
tcp.flags.syn == 1 && tcp.flags.ack == 0 # show SYN-only packets (Nmap -sS)

Filters are like panning the camera to focus only on suspicious activity.

---

### 3) Simulate Normal Traffic (Victim — Ubuntu)
**On Ubuntu (Victim):**
- Open Firefox and visit: `https://www.google.com`  
  *(Or run: `curl -I https://www.google.com` in terminal)*

**In Wireshark:** apply:

ip.addr == 192.168.1.xxx && http


**You’ll see:**
- `GET /canonical.html HTTP/1.1` (or similar GET requests)
- `HTTP/1.1 200 OK`

This gives a baseline of normal browsing behavior for comparison.

---

### 4) Simulate an Attack from Kali (Attacker)

#### A) Ping Flood (DoS)
**On Kali (Attacker):**

ping -f 192.168.1.xxx
### Press Ctrl+C to stop when you've captured enough

In Wireshark: apply:

*icmp && ip.addr == 192.168.1.xxx*

You’ll see:

`Repeated ICMP Echo Request packets from Kali to Ubuntu.`

`Echo Replies if the victim responds.`

`Packet rate spike; possible packet drops if victim is overwhelmed.`

This simulates a denial-of-service using ICMP.

B) Nmap SYN Scan (stealth probe)

On Kali (Attacker):

`sudo nmap -sS 192.168.1.xxx`

### Optional reachability check:
`ping -c 5 192.168.1.xxx`


`sudo nmap -sS 192.168.1.xxx`
### Optional reachability check:
`ping -c 5 192.168.1.xxx`


In Wireshark: apply:

`tcp.flags.syn == 1 && tcp.flags.ack == 0 && ip.dst == 192.168.1.xxx`


You’ll see:

`TCP SYN packets from Kali to many destination ports (e.g., 22, 80, 443).`

`For open ports: SYN/ACK responses; for closed ports: RST replies.`

**Analogy: like someone checking which doors are unlocked.***


### 5) Quick Analysis Tips & Tools in Wireshark

- Use Statistics → Conversations to find top talkers (which IPs are busiest).

- Use Statistics → IO Graph to visualize packet/sec spikes (useful during DoS).

- Right-click a packet → Follow → TCP Stream to view full session payloads (HTTP, SSH, etc.).

- Save captures: File → Save As (e.g., phase1_capture.pcapng).


<img src="pic/fem2.png" alt="Wireshark normal traffic screenshot" />


The screenshot above displays the normal traffic on wireshark on kali with no filters to show there was an internet connection.

<div style="background-color: #03182cff; padding: 20px; border-radius: 5px; border-left: 5px solid #2E86C1;">

# <span style="color: #2E86C1">🔍 Analyze: DNS Traffic + ICMP & Nmap Analysis</span>

<div style="background-color: #EBF5FB; padding: 15px; border-radius: 3px; margin: 10px 0;">
<strong>Goal:</strong> Inspect DNS queries/responses, review the previously run ping-flood (ICMP DoS) capture, and correlate with the Nmap stealth scan.
</div>

## <span style="color: #2471A3">💻 1) DNS Traffic — Filter & What to Expect</span>

<div style="background-color: #; color: #ffffff; padding: 10px; border-radius: 3px; font-family: 'Consolas';">
<strong>Display filter (replace `192.168.1.xxx` with the victim IP):</strong>
<pre>ip.addr == 192.168.1.xxx && dns</pre>
</div>

<div style="padding-left: 20px; border-left: 3px solid #85C1E9; margin: 15px 0;">
You'll see:
- 📝 DNS queries such as A www.google.com (the client asking "what is the IP for this hostname?")
- 💡 DNS responses from the DNS server (e.g., 192.168.1.1) containing the resolved IP addresses
- 🔑 Note: DNS is like asking for directions — nearly every website visit starts with a DNS query.
</div>

<figure>
<img src="pic/fem3.png" alt="DNS traffic screenshot" style="border: 2px solid #ddd; border-radius: 5px; max-width: 100%; height: auto;"/>
<figcaption style="text-align: center; font-style: italic; color: #666;">DNS queries and responses captured for the victim host</figcaption>
</figure>

## <span style="color: #2471A3">⚡ 2) Ping Flood (ICMP DoS) Analysis</span>

<div style="background-color: #a28815ff; padding: 15px; border-radius: 3px; margin: 10px 0;">
A ping flood was run from the attacker to the victim to simulate a Denial of Service. Use the icmp filter to isolate ping traffic and confirm the flood.
</div>

<div style="background-color: #; color: #f1f0edff; padding: 10px; border-radius: 3px; font-family: 'Consolas';">
<strong>Command used on Kali (Attacker):</strong>
<pre>ping -f 192.168.1.xxx</pre>

<strong>Wireshark filter to inspect ICMP:</strong>
<pre>icmp</pre>
</div>

<div style="padding-left: 20px; border-left: 3px solid #E74C3C; margin: 15px 0;">
<strong>In Wireshark you will observe:</strong>
- 🔄 Repeated ICMP Echo Request frames from attacker → victim
- ↩️ Corresponding Echo Reply frames from victim → attacker (if the victim responds)
- ⚡ Rapid packet intervals (very small delta times) and a pronounced packet-rate spike
</div>

<figure>
<img src="pic/fem5.png" alt="ICMP echo request/reply screenshot" style="border: 2px solid #ddd; border-radius: 5px; max-width: 100%; height: auto;"/>
<figcaption style="text-align: center; font-style: italic; color: #666;">ICMP Echo Request/Reply pattern during ping flood</figcaption>
</figure>

<div style="background-color: #a28815ff; padding: 15px; border-radius: 3px; margin: 10px 0;">
<strong>What we're seeing in sample frames:</strong>
- 📊 Echo Request (e.g., Frame 485) and Echo Reply (e.g., Frame 486) pairs
- 🔢 Sequence numbers and payload sizes (e.g., seq=1/256, seq=2/512)
- ⏱️ ttl values (e.g., ttl=64) and ICMP id (e.g., id=0x0001) for flow identification
</div>

</div>

<h2 style="color: #2471A3; font-size:1.6em; margin-top:1em;">3) Nmap Stealth Scan — What was run & how to spot it</h2>

<div style="background-color: #; border-left:5px solid #efda86ff; padding:1em; margin-bottom:1em;">
  <h3 style="margin-top:0; color:#2471A3;">🔧 Command used on Kali (Attacker):</h3>
  <pre style="background-color:#eef; padding:0.8em; border-radius:6px; font-size:1em;">sudo nmap -sS -p- 192.168.1.xxx</pre>

  <ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
    <li><code>-sS</code>: SYN (half-open/stealth) scan — sends SYN but does not complete handshake</li>
    <li><code>-p-</code>: scan all TCP ports (1–65535)</li>
  </ul>
</div>

<h3 style="color:#2471A3;">🔍 Wireshark filters to identify Nmap activity (replace IPs accordingly):</h3>
<pre style="background-color:#b99c18ff; padding:1em; border-radius:6px; font-size:0.95em;">
### SYNs sent by attacker (probe packets)
tcp.flags.syn == 1 && tcp.flags.ack == 0 && ip.src == 192.168.1.xx

- SYN-ACK responses (open ports) from victim
tcp.flags.syn == 1 && tcp.flags.ack == 1 && ip.dst == 192.168.1.xx
- RST (reset) responses indicating closed ports
tcp.flags.reset == 1
</pre>

<h3 style="color:#2471A3;">📈 What you’ll see in captures:</h3>
<ul style="list-style-type:circle; padding-left:1.2em; font-size:1em;">
  <li>Many TCP SYN packets from attacker to different destination ports on the victim.</li>
  <li>SYN/ACK responses for open ports (example: 22/tcp open ssh) — Nmap reports these as open and typically sends an RST afterward to avoid completing the handshake.</li>
  <li>RST replies for closed ports (visible as red lines in the default Wireshark color scheme).</li>
</ul>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem4.png" alt="Nmap and HTTP filter screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em;">Caption: Filters used to visualize traffic (example shows HTTP view when the victim visited Google; same filter strategy applies to SYN/SYN-ACK/RST during scans).</p>
</div>

<h2 style="color:#2471A3; font-size:1.6em; margin-top:1em;">4) Quick correlation steps (timeline & evidence)</h2>
<ul style="list-style-type:square; padding-left:1.2em; font-size:1em;">
  <li>Note timestamps when <code>ping -f</code> and <code>nmap -sS -p-</code> were started (use terminal timestamps or shell history).</li>
  <li>In Wireshark, use <strong>Statistics → IO Graph</strong> to visualize the ICMP flood spike and <strong>Statistics → Conversations</strong> to identify top talkers (attacker IP should appear at top).</li>
  <li>Filter for SYN packets and locate SYN/ACK frames to confirm open ports (record frame numbers for screenshots).</li>
  <li>Save filtered PCAP excerpts for evidence: <strong>File → Export Specified Packets</strong> (choose displayed packets only).</li>
</ul>

<div style="background-color:#fff3cd; border-left:5px solid #f1c40f; padding:1em; margin-top:1em;">
  <strong>⚠️ Safety reminder:</strong> Only run these tests in an isolated lab or against systems you own/have explicit permission to test.
</div>

<h3 style="color:#2471A3; margin-top:1.5em;">✅ Checklist before running the steps above:</h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Replace <code>192.168.1.xxx</code> and <code>192.168.1.xx</code> with the actual victim and attacker IPs.</li>
  <li>Start Wireshark capture on the victim before launching attacker commands.</li>
  <li>Record timestamps and save PCAPs for later correlation.</li>
</ul>

<h2 style="color:#4c9e80; font-size:1.6em; margin-top:1em;"> Nmap Stealth Scan Analysis</h2>

<h3 style="color:#2471A3;">🛠️ <strong>Command used (Attacker — Kali):</strong></h3>
<pre style="background-color:#; padding:1em; border-radius:6px; font-size:1em;">
sudo nmap -sS -p- 192.168.1.xxx
-sS : Stealth SYN scan (sends SYN but does not complete the TCP handshake)
-p- : Scan all TCP ports (1–65535)
</pre>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem6.png" alt="Nmap SYN scan screenshot" style="max-width:100%; border-radius:8px;">
</div>

<h3 style="color:#2471A3;">📋 <strong>What the scan does (quick overview)</strong></h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Sends a TCP SYN to each target port.</li>
  <li>If the port is open, the target replies with SYN/ACK.</li>
  <li>Nmap typically sends an RST afterward to avoid completing the handshake (keeps the scan stealthier).</li>
  <li>If the port is closed, the target replies with RST (or RST/ACK).</li>
  <li>If the port is filtered, there may be no reply or an ICMP unreachable.</li>
</ul>

<p style="font-style:italic;">This stealthy behavior can evade some logging or simple defenses because the full TCP connection is never completed.</p>

<h3 style="color:#2471A3;">🔎 <strong>Wireshark filters</strong></h3>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
tcp.flags.syn == 1 && tcp.flags.ack == 0      # SYN probes sent by scanner
tcp.flags.syn == 1 && tcp.flags.ack == 1      # SYN-ACK responses (open ports)
tcp.flags.reset == 1 && tcp.flags.ack == 1    # RST (closed ports)
</pre>

<h3 style="color:#2471A3;">📡 <strong>What you’ll see in the capture</strong></h3>
<ul style="list-style-type:circle; padding-left:1.2em; font-size:1em;">
  <li>Lots of SYN packets from attacker → victim probing many destination ports.</li>
  <li>SYN/ACK responses for any open ports (example: 22/tcp open ssh).</li>
  <li>RST replies for closed ports (often shown in red by Wireshark's default coloring).</li>
</ul>

<h4 style="color:#;">🔬 Example RST line observed in captures:</h4>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">[RST, ACK] Seq=1 Ack=1 Win=0 Len=0</pre>

<h4 style="color:#;">🔬 Example SYN-ACK line observed in captures:</h4>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">60 22 - 38840 [SYN, ACK] Seq=0 Ack=1 Win=64240 Len=0 MSS=1460</pre>

<p style="font-style:italic;">This indicates port 22 (SSH) is open and the host responded with SYN-ACK.</p>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem7.png" alt="Nmap results screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em;">Caption: Packet list showing SYN probes and the SYN-ACK that indicates SSH (port 22) is open. Closed ports appear with RST replies (red).</p>
</div>

<h3 style="color:#2471A3;">🧠 <strong>Interpretation</strong></h3>
<ul style="list-style-type:square; padding-left:1.2em; font-size:1em;">
  <li>The scan probed all 65,535 TCP ports and completed in approximately 174 seconds.</li>
  <li>Only one open service was detected: SSH on port 22.</li>
  <li>All other ports returned RST (closed).</li>
  <li>Behavior is consistent with a typical SYN scan: attacker sends SYNs, then interprets the responses to determine port state without completing connections.</li>
</ul>

<h3 style="color:#2471A3;">🧭 <strong>Useful Wireshark view to show the TCP interactions between attacker & victim</strong></h3>
<p>Use a combined filter to restrict display to the attacker <strong>&lt;→&gt;</strong> victim conversation and SYN/RST events (replace IPs as needed):</p>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
ip.src == 192.168.1.20 && ip.dst == 192.168.1.10 && (tcp.flags.syn == 1 || tcp.flags.reset == 1)
</pre>
<p style="font-style:italic;">This highlights the probe packets and the target's responses; red rows commonly show closed ports (RST).</p>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem8.png" alt="TCP handshake and RST visualization" style="max-width:100%; border-radius:8px;">
</div>

<h3 style="color:#2471A3;">📝 <strong>Quick evidence & reporting tips</strong></h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Record timestamps for when <code>nmap</code> started/stopped (shell history or terminal timestamps).</li>
  <li>Save a filtered PCAP with only the relevant packets: <strong>File → Export Specified Packets</strong> (displayed packets). Suggested filename: <code>nmap_stealth_scan_192.168.1.10.pcapng</code>.</li>
  <li>Include frame numbers of representative SYN, SYN/ACK and RST packets in your report (e.g., the frame showing the SYN-ACK on port 22).</li>
  <li>Note the scan duration (~174 seconds) and that only SSH was reachable — useful for risk prioritization (SSH hardening, rate-limiting, geo-blocking, or MFA).</li>
</ul>

<h2 style="color:#; font-size:1.6em; margin-top:1em;"> Bonus: <code>hping3</code> Command (SYN probes / lightweight SYN flood)</h2>

<h3 style="color:#2471A3;">🛠️ <strong>Command (Attacker — Kali):</strong></h3>
<pre style="background-color:#eef; padding:1em; border-radius:6px; font-size:1em;">
sudo hping3 -c 100 -S -p 80 192.168.1.xxx
</pre>

<h4 style="color:#;">🔍 Breakdown:</h4>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li><code>-c 100</code> — send 100 packets</li>
  <li><code>-S</code> — set the SYN flag (simulate a TCP connection request)</li>
  <li><code>-p 80</code> — target port 80 (HTTP)</li>
  <li><code>192.168.1.xxx</code> — victim IP (replace with your lab victim IP)</li>
</ul>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem9.png" alt="hping3 SYN packets screenshot" style="max-width:100%; border-radius:8px;">
</div>

<h3 style="color:#2471A3;">📋 <strong>What this simulates / why use it:</strong></h3>
<ul style="list-style-type:circle; padding-left:1.2em; font-size:1em;">
  <li>Sends 100 SYN packets to port 80 on the target — can simulate a SYN flood or be used for focused stealthy probing.</li>
  <li>Useful to test firewall rules, IDS/IPS detection, and how servers handle a burst of connection attempts.</li>
</ul>

<h3 style="color:#2471A3;">🔎 <strong>What to look for in Wireshark / Logs:</strong></h3>
<ul style="list-style-type:square; padding-left:1.2em; font-size:1em;">
  <li>Many TCP packets with the SYN flag set from attacker → victim.</li>
  <li>If the port is open: the victim may reply with SYN/ACK (then attacker may or may not respond).</li>
  <li>If closed: the victim may reply with RST.</li>
  <li>IDS/Firewall logs may show repeated connection attempts or trigger rate/scan rules.</li>
</ul>

# <h2 style="color:#2471A3; font-size:1.6em; margin-top:1em;"> How to Analyze in Wireshark — SYN to HTTP (port 80)</h2>

<h3 style="color:">🎯 <strong>Goal:</strong></h3>
<p style="font-size:1em;">Verify SYN packets sent to port 80, observe victim responses (SYN/ACK or RST), and confirm whether HTTP service is listening.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🔍 <strong>1) Apply this Wireshark display filter (replace <code>192.168.1.xxx</code>)</strong></h3>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
ip.src == 192.168.1.xxx && tcp.dstport == 80 && tcp.flags.syn == 1
</pre>
<p style="font-size:1em;">This shows SYN packets originating from the attacker to destination port 80 on the victim.</p>
<p style="font-style:italic; color:#;">Use this first to confirm the attacker sent SYNs targeting HTTP.</p>

<h3 style="color:#;">📈 <strong>2) What to expect / how to interpret responses</strong></h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>If the victim replies with SYN, ACK → port open (service listening).</li>
  <li>If the victim replies with RST or RST, ACK → port closed (no service).</li>
  <li>No response or ICMP unreachable → port filtered by firewall or network device.</li>
</ul>

<h3 style="color:#;">🧪 <strong>3) Lab findings (summary)</strong></h3>
<ul style="list-style-type:circle; padding-left:1.2em; font-size:1em;">
  <li>A SYN flood was simulated against HTTP (port 80) using hping3 (100 SYN packets).</li>
  <li>The capture showed attacker SYNs to port 80; the victim consistently replied with RST (or RST, ACK) packets.</li>
  <li><strong>Interpretation:</strong> port 80 is closed on the victim — no HTTP service listening.</li>
  <li>This matched the earlier <code>nmap -sS -p-</code> results where port 80 also reported closed.</li>
</ul>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem10.png" alt="SYN to port 80 analysis screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em; color:#;">Caption: Wireshark view showing SYN probes to port 80 and RST replies from the target (indicating closed port).</p>
</div>

<h3 style="color:#;">🧰 <strong>4) Additional filters you may apply (single-use examples)</strong></h3>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
# Show SYN-ACK responses (open ports)
tcp.flags.syn == 1 && tcp.flags.ack == 1

# Show RST responses (closed ports)
tcp.flags.reset == 1 && tcp.flags.ack == 1
</pre>
<p style="font-style:italic; ;">Apply one of the above after confirming SYNs to view how the victim answered.</p>

# <h2 style="color:#2471A3; font-size:1.6em; margin-top:1em;">Phase 2 — pfSense & Snort: Firewall and Policy Enforcement</h2>

<h3 style="color:#;">🎯 <strong>Goal:</strong></h3>
<p style="font-size:1em;">Block malicious traffic and detect intrusions using Snort on pfSense.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🎯 <strong>Objective</strong></h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Block known ransomware IPs (e.g., <strong>Medusa</strong>, <strong>Lockbit</strong>)</li>
  <li>Reject <strong>ICMP traffic</strong> (to mitigate ping floods)</li>
  <li>Monitor and enforce <strong>firewall rules</strong></li>
  <li>Integrate <strong>Snort</strong> for intrusion detection and alerting</li>
</ul>


<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🖥️ <strong>1) Access pfSense Web Interface</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Open your web browser.</li>
  <li>Navigate to:<br><code>https://192.168.1.x</code></li>
</ol>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">(Replace with your pfSense LAN IP address)</pre>
<ol start="3" style="padding-left:1.2em; font-size:1em;">
  <li>Log in using your credentials:<br>
    <strong>Username:</strong> <code>xxxxx</code><br>
    <strong>Password:</strong> <code>xxxxxx</code>
  </li>
</ol>
<p style="font-style:italic; color:#;">💡 Tip: Think of pfSense as your organization’s gatekeeper—it decides who can come in and who gets blocked.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🌐 <strong>2) Configure Network Interfaces</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Assign your <strong>LAN</strong> and <strong>WAN</strong> interfaces.</li>
  <li>Set the LAN IP to your internal subnet, e.g.:<br><code>192.168.1.xxx</code></li>
</ol>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">Verify you can reach the pfSense dashboard via your LAN(1. Open your web browser (e.g., Chrome, Firefox).
2. In the address bar, enter your pfSense LAN IP).</pre>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

`Snort watches for suspicious patterns and blocks them automatically.`

<h3 style="color:#;">🛡️ <strong>3) Install Snort (Intrusion Detection System)</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>From the top menu, go to:<br><code>System → Package Manager → Available Packages</code></li>
  <li>Search for <strong>Snort</strong>.</li>
  <li>Click <strong>Install</strong> and wait for the installation to complete.</li>
</ol>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🔧 <strong>4) Configure Snort</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to:<br><code>Services → Snort</code></li>
</ol>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">
Add a new interface — select LAN  
Enable Snort on that interface  
Under Categories, enable:  
- Emerging Threats  
- Malware  
- Policy rules  
Check Block Offenders to automatically drop malicious traffic  
Click Save and Apply Changes
</pre>
<p style="font-style:italic; color:#;">🔍 Snort acts like a surveillance guard—it reads every packet and raises an alarm when it recognizes suspicious behavior.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🚧 <strong>5) Create Firewall Rules</strong></h3>
<p>Navigate to:<br><code>Firewall → Rules → LAN</code></p>
<pre style="background-color:#f4f4f4; padding:0.8em; border-radius:6px; font-size:0.95em;">
Add rules to:  
- Block ICMP (ping traffic)  
- Block known malicious IPs (ransomware groups)
</pre>

Verify:
- Try pinging Ubuntu from Kali again
- Check Status > System Logs > Firewall for blocked attempts

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🧩 <strong>Create Aliases for Ransomware IPs</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to:<br><code>Firewall → Aliases</code></li>
  <li>Click <strong>Add</strong>.</li>
  <li>Enter the following details:</li>
</ol>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
Name: Medusa_Ransomware  
Type: Host(s)  
IPs:  
185.220.101.1  
185.220.101.2
</pre>
<p>Click <strong>Save</strong> → <strong>Apply Changes</strong>.</p>
<p>Repeat for: <strong>Lockbit_Ransomware</strong> with its respective IPs.</p>
<p style="font-style:italic; color:#;"> Think of Aliases like contact groups in your phone—they help you block or allow entire sets of IPs easily.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🚫 <strong>6) Block Ransomware IPs on Port 80</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to:<br><code>Firewall → Rules → WAN</code></li>
  <li>Click <strong>Add</strong>.</li>
  <li>Set:</li>
</ol>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
Action: Block  
Interface: WAN  
Protocol: TCP  
Source: Medusa_Ransomware alias  
Destination: WAN address  
Destination Port: 80 (HTTP)  
Description: Block Medusa ransomware IPs on port 80
</pre>
<p>Click <strong>Save</strong> → <strong>Apply Changes</strong>.</p>
<p>Repeat the same for the <strong>Lockbit_Ransomware</strong> alias.</p>
<p style="font-style:italic; color:#;">✅ This prevents ransomware from “phoning home” to its command server via HTTP.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">📵 <strong>7) Reject ICMP (Ping) Traffic</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to:<br><code>Firewall → Rules → WAN</code></li>
  <li>Click <strong>Add</strong>.</li>
  <li>Configure:</li>
</ol>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
Action: Reject  
Protocol: ICMP  
Source: any  
Destination: any  
Description: Reject ICMP protocol
</pre>
<p>Click <strong>Save</strong> → <strong>Apply Changes</strong>.</p>
<p style="font-style:italic; color:#;">✅ This blocks ping requests, stopping attackers from scanning or flooding your network.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#

# <h2 style="color:#2471A3; font-size:1.6em; margin-top:1em;">pfSense: Firewall Rules, Snort IDS, and IP Alias Management</h2>

<p style="font-size:1em;">📝The following shows how pfSense was configured to block known ransomware groups (<strong>Lockbit</strong>, <strong>Medusa</strong>, <strong>Phobos</strong>) on port 80, reject ICMP, and enable Snort for IDS/IPS.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🛡️ <strong>Firewall rules overview</strong></h3>
<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem11.png" alt="Firewall rules blocking ransomware and ICMP" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em; color:#;">Caption: Firewall rules that block Lockbit & Medusa IPs on port 80 and reject ICMP (ping).</p>
</div>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🔧 <strong>Enable Snort (IDS/IPS)</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to: <code>System → Package Manager → Available Packages</code></li>
  <li>Search for <strong>Snort</strong> and click <strong>Install</strong>.</li>
  <li>After installation: <code>Services → Snort</code></li>
  <li>Add an interface (e.g., <strong>WAN</strong>) and <strong>Enable Snort</strong>.</li>
  <li>Enable rule sets (recommended):<br>
    - Emerging Threats<br>
    - Snort GPLv2 Community Rules
  </li>
  <li>Enable <strong>Block Offenders</strong> (auto-block) → <strong>Save</strong> → <strong>Apply Changes</strong></li>
</ol>
<p style="font-style:italic; color:#;">💡 Snort monitors traffic for suspicious patterns and (when enabled) can automatically block offenders.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">📦 <strong>Create an Alias for Lockbit (example)</strong></h3>
<p>Use aliases to group threat IPs for easier rule creation.</p>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to: <code>Firewall → Aliases</code> → <strong>Add</strong></li>
  <li>Fill fields:</li>
</ol>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
Name: Lockbit_ransomware  
Type: Host(s)  
Description: Blocks Lockbit ransomware IPs  
Entries:  
3.64.4.198  
3.67.161.133  
185.215.113.66  
13.107.4.52  
92.51.2.22
</pre>
<p><strong>Save</strong> → <strong>Apply Changes</strong></p>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem12.png" alt="Lockbit alias list screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em; color:#;">Caption: Alias listing showing Lockbit IPs grouped under <code>Lockbit_ransomware</code>.</p>
</div>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🚫 <strong>Firewall rule to block Lockbit on port 80</strong></h3>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Go to: <code>Firewall → Rules → WAN</code> → <strong>Add</strong></li>
  <li>Configure:</li>
</ol>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px; font-size:0.95em;">
Action: Block  
Interface: WAN  
Protocol: TCP  
Source: Lockbit_ransomware (alias)  
Destination: WAN address  
Destination port range: 80 (HTTP)  
Description: Block Lockbit ransomware IPs on port 80
</pre>
<p><strong>Save</strong> → <strong>Apply Changes</strong></p>
<p style="font-style:italic; color:#;">✅ This prevents HTTP-based command-and-control or payload delivery from Lockbit IPs.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">📚 <strong>IP Alias Collections (blocked threat groups)</strong></h3>
<p>We created three aliases grouping known threat IPs:</p>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li><strong>Lockbit_ransomware</strong><br><code>3.64.4.198</code>, <code>3.67.161.133</code>, <code>185.215.113.66</code>, <code>13.107.4.52</code>, <code>92.51.2.22</code></li>
  <li><strong>Medusa_Ransomware</strong><br><code>18.158.58.205</code>, <code>18.197.239.109</code></li>
  <li><strong>Phobos_ransomware</strong><br><code>194.165.16.4</code>, <code>45.9.74.14</code>, <code>147.78.47.224</code>, <code>185.202.0.111</code></li>
</ul>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem13.png" alt="Alias groups screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em; color:#;">Caption: Aliases grouped by threat family for efficient management.</p>
</div>

### Firewall Rules Summary (applied)
<img src="pic/fem14.png" alt="Firewall rules & GeoIP blocking screenshot" />

*Caption:* Rules reduce exposure by blocking known threat IPs, rejecting risky protocols, and blocking SSH on WAN.

| Rule Name | Protocol | Source | Destination | Port | Action | Purpose |
|-----------|----------|--------|-------------|------|--------|---------|
| Block Lockbit | TCP | `Lockbit_ransomware` | WAN address | 80 (HTTP) | Block | Prevent web-based C2 / payload delivery |
| Block Medusa | TCP | `Medusa_Ransomware` | WAN address | 80 (HTTP) | Block | Block Medusa IPs |
| Block Phobos | TCP | `Phobos_ransomware` | WAN address | 80 (HTTP) | Block | Block Phobos IPs |
| Reject ICMP | ICMP | any | any | — | Reject | Stop ping floods & reconnaissance |
| Block SSH (WAN) | TCP | any | WAN address | 22 (SSH) | Block | Prevent remote login attempts from WAN |


---

### GeoIP Blocking (optional)
- pfSense can be configured with GeoIP/block lists to restrict inbound traffic from high-risk countries or continents.  
- This reduces attack surface but may block legitimate traffic—use carefully and whitelist known good ranges.

<h2 style="color:#2471A3; font-size:1.6em; margin-top:1em;">## GeoIP Filtering in pfSense</h2>

<h3 style="color:#;">🎯 <strong>Objective:</strong></h3>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Block inbound traffic from high-risk countries</li>
  <li>Allow access only from trusted geographic regions (e.g., North America / Lab network)</li>
</ul>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🛠️ <strong>Configuration Overview</strong></h3>
<p style="font-size:1em;">
Using pfSense’s <strong>GeoIP filtering</strong> feature, inbound traffic from selected continents or countries was blocked.<br>
Only trusted regions (such as North America) were allowed to communicate with the lab environment.
</p>

<h4 style="color:#2471A3;">📋 <strong>Steps:</strong></h4>
<ol style="padding-left:1.2em; font-size:1em;">
  <li>Install the <strong>pfBlockerNG-devel</strong> package<br>
      Navigate to: <code>System → Package Manager → Available Packages</code><br>
      Search <strong>pfBlockerNG-devel</strong> → <strong>Install</strong>
  </li>
  <li>Go to: <code>Firewall → pfBlockerNG → IP</code></li>
  <li>Enable <strong>GeoIP</strong> filtering</li>
  <li>Choose the continents or countries to block (e.g., Africa, Asia, Eastern Europe)</li>
  <li>Set <strong>Action:</strong> <em>Deny Both (Inbound/Outbound)</em></li>
  <li>Under <strong>Alias Type</strong>, choose “Country”</li>
  <li><strong>Save</strong> and <strong>Apply Changes</strong></li>
</ol>

<p style="font-style:italic; color:#;">This configuration blocks unsolicited inbound traffic from entire regions, dramatically reducing the network’s attack surface.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🖼️ <strong>Screenshot — GeoIP Filtering</strong></h3>
<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem15.png" alt="GeoIP filtering configuration screenshot" style="max-width:100%; border-radius:8px;">
  <p style="font-style:italic; font-size:0.95em; color:#;">Caption: GeoIP filtering enabled to restrict inbound traffic from high-risk regions and allow only trusted zones such as North America.</p>
</div>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">📈 <strong>Results & Impact</strong></h3>
<ul style="list-style-type:circle; padding-left:1.2em; font-size:1em;">
  <li>Inbound traffic from untrusted countries is automatically rejected.</li>
  <li>The <strong>attack surface</strong> is significantly minimized, as external probes or malicious IPs from excluded regions cannot reach the network.</li>
  <li>Legitimate internal or lab-based activity remains unaffected.</li>
</ul>

<p style="font-style:italic; color:#;"><strong>Key takeaway:</strong> GeoIP filtering acts as an additional perimeter defense—ideal for labs or internal systems where legitimate access is geographically limited.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🧩 <strong>Notes on Overall Firewall & IDS Configuration</strong></h3>
<table style="width:100%; border-collapse:collapse; font-size:0.95em; margin-top:1em; border:1px solid #ccc;">
  <thead style="background-color:#f0f0f0;">
    <tr>
      <th style="border:1px solid #ccc; padding:0.6em;">Security Feature</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Description</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Purpose</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;"><strong>IP Aliases (Lockbit, Medusa, Phobos)</strong></td>
      <td style="border:1px solid #ccc; padding:0.6em;">Grouped verified malicious IPs for easy blocking</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Prevent ransomware-related HTTP traffic on port 80</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;"><strong>ICMP Blocking</strong></td>
      <td style="border:1px solid #ccc; padding:0.6em;">Rejected ping floods & reconnaissance attempts</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Prevent discovery & DoS</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;"><strong>SSH Blocking (WAN)</strong></td>
      <td style="border:1px solid #ccc; padding:0.6em;">Stopped brute-force attempts on port 22</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Harden remote access</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;"><strong>Snort IDS/IPS</strong></td>
      <td style="border:1px solid #ccc; padding:0.6em;">Detected & blocked intrusion attempts in real time</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Deep packet inspection & proactive response</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;"><strong>GeoIP Filtering</strong></td>
      <td style="border:1px solid #ccc; padding:0.6em;">Blocked high-risk countries / continents</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Reduced global threat exposure</td>
    </tr>
  </tbody>
</table>

<p style="font-size:1em; margin-top:1em;">The <strong>firewall state table</strong> confirmed all rules were actively enforced and functioning as intended.</p>

<hr style="border:none; border-top:1px solid #ccc; margin:1em 0;">

<h3 style="color:#;">🧠 <strong>Final Reflection</strong></h3>
<p style="font-size:1em;">
Through layered defense with <strong>pfSense + Snort + GeoIP</strong>, we built a <strong>proactive and scalable firewall strategy</strong>:
</p>
<ul style="list-style-type:disc; padding-left:1.2em; font-size:1em;">
  <li>Not just reacting to threats, but <strong>anticipating and neutralizing</strong> them before they reach the network.</li>
  <li>Reduced exposure to known ransomware IPs and high-risk global regions.</li>
  <li>Maintained a clean, controlled environment suitable for secure lab operations.</li>
</ul>

<p style="font-style:italic; color:#;"><strong>Result:</strong> A hardened perimeter with minimal attack surface, active intrusion detection, and geographic access control.</p>

<h2 style="color:#2471A3; font-size:1.6em;">🛡️ Testing Firewall Rules and IDS Functionality</h2>

<h3 style="color:#;">📡 ICMP Rule Test</h3>
<p>To verify the effectiveness of our <strong>ICMP blocking rule</strong>, we tested connectivity by attempting to ping the host network.</p>

<h4 style="color:#2471A3;">🔧 Command Executed:</h4>
<pre style="background-color:#eef; padding:1em; border-radius:6px;">
ping -c4 192.168.1.xxx
</pre>

<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem16.png" alt="Firewall blocking ICMP: ping test results showing 100% packet loss" style="max-width:100%; border-radius:8px;">
</div>

<h4 style="color:#2471A3;">🔍 Observation:</h4>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px;">
From 192.168.1.xxx icmp_seq=1 Destination Host Unreachable
4 packets transmitted, 0 received, +4 errors, 100% packet loss
</pre>

<h4 style="color:#2471A3;">🧠 Interpretation:</h4>
<ul style="font-size:1em;">
  <li>The ICMP reject rule is working correctly.</li>
  <li>The firewall actively drops all ping (ICMP) packets.</li>
  <li>This prevents reconnaissance and DoS attempts leveraging ICMP.</li>
</ul>
<p style="font-style:italic; color:#;">✅ Result: 100% packet loss confirms that the firewall is successfully rejecting ping traffic.</p>

<hr>

<h3 style="color:#;">🌍 GeoIP Filtering Verification</h3>
<p>The <strong>pfBlockerNG</strong> module was configured to enforce GeoIP-based filtering.</p>
<ul style="font-size:1em;">
  <li>Inbound traffic from high-risk regions—including Africa, Asia, Europe, Oceania, and South America—was denied.</li>
  <li>This significantly reduced the attack surface by restricting inbound traffic to only trusted geographic zones (e.g., North America).</li>
</ul>
<p style="font-style:italic; color:#;">🌍 By denying entire continents, we reduced exposure to widespread botnets, ransomware C2 servers, and brute-force campaigns.</p>

<hr>

<h3 style="color:#;">🧩 Layered Defense Summary</h3>
<table style="width:100%; border-collapse:collapse; font-size:0.95em; border:1px solid #ccc;">
  <thead style="background-color:#f0f0f0;">
    <tr>
      <th style="border:1px solid #ccc; padding:0.6em;">Defense Layer</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Tool</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Objective</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Result</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">IP Blocking</td>
      <td style="border:1px solid #ccc; padding:0.6em;">pfSense</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Block known ransomware IPs (Lockbit, Medusa, Phobos)</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Effective</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">Protocol Control</td>
      <td style="border:1px solid #ccc; padding:0.6em;">pfSense</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Reject ICMP (ping floods)</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ 100% packet loss confirmed</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">GeoIP Filtering</td>
      <td style="border:1px solid #ccc; padding:0.6em;">pfBlockerNG</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Block traffic from high-risk regions</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Confirmed effective</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">Intrusion Detection</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Snort</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Detect & block scan/flood activity</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Alerts generated</td>
    </tr>
  </tbody>
</table>

<hr>

<h4 style="color:#2471A3;">🧠 Combined Strategy:</h4>
<ul style="font-size:1em;">
  <li><strong>IP-based filtering</strong> (ransomware groups)</li>
  <li><strong>Protocol blocking</strong> (ICMP, SSH)</li>
  <li><strong>Geographic restriction</strong> (GeoIP)</li>
  <li><strong>Behavioral detection</strong> (Snort IDS)</li>
</ul>
<p style="font-style:italic; color:#;">Together, these layers form a resilient, proactive defense posture for lab and production environments.</p>

<h2 style="color:#2471A3; font-size:1.6em;">🧪 Snort Intrusion Detection Test</h2>

<p>Snort was installed and configured to detect and block intrusion attempts in real time.</p>

<h3 style="color:#;">⚙️ Configuration Steps Recap</h3>
<ol style="font-size:1em; padding-left:1.2em;">
  <li>Navigate to: <code>System → Package Manager → Available Packages</code></li>
  <li>Search for <strong>Snort</strong> → Click <strong>Install</strong></li>
  <li>Go to: <code>Services → Snort</code></li>
  <li>Add Interface: <strong>WAN</strong></li>
  <li>Enable Snort</li>
  <li>Select Rule Sets:
    <ul>
      <li>Emerging Threats</li>
      <li>Snort GPLv2 Community Rules</li>
    </ul>
  </li>
  <li>Enable <strong>Block Offenders</strong></li>
  <li>Click <strong>Save</strong> and <strong>Apply</strong></li>
</ol>

<p>Once active, Snort monitors all incoming/outgoing traffic for known attack signatures.</p>

<h3 style="color:#;">📊 Snort Detection in Action</h3>
<div style="text-align:center; margin:1em 0;">
  <img src="pic/fem17.png" alt="Snort alert panel showing detected Nmap scans and ICMP floods" style="max-width:100%; border-radius:8px;">
</div>

<h4 style="color:#2471A3;">🔍 What Happened</h4>
<ul style="font-size:1em;">
  <li>Snort identified multiple Nmap scan and ICMP flood attempts.</li>
  <li>Alerts were raised within pfSense in real time.</li>
  <li>Offending IPs were automatically blocked thanks to the <strong>Block Offenders</strong> feature.</li>
</ul>
<p style="font-style:italic; color:#;">🚨 Insight: Snort acts as a dynamic IDS/IPS—detecting suspicious activity and blocking offenders on the spot.</p>

<hr>

<h2 style="color:#2471A3; font-size:1.6em;">✅ Phase 2 Summary</h2>

<ul style="font-size:1em;">
  <li>Blocked ransomware IPs (<strong>Lockbit</strong>, <strong>Medusa</strong>, <strong>Phobos</strong>)</li>
  <li>Rejected <strong>ICMP</strong> & <strong>SSH</strong> traffic</li>
  <li>Applied <strong>GeoIP filtering</strong> to block high-risk regions</li>
  <li>Verified <strong>firewall and IDS rules</strong> through live testing</li>
</ul>

<p style="font-style:italic; color:#;"><strong>🧠 Key takeaway:</strong> This layered defense approach merges <strong>preventive controls</strong> (firewall) and <strong>detective controls</strong> (IDS) for a resilient, proactive security posture.</p>

<hr>

<h2 style="color:#2471A3; font-size:1.6em;">⏭️ Next Phase → Wazuh: Security Event Monitoring & Response</h2>

<p>With our firewall and IDS layers tested, we’ll now move to <strong>Phase 3 – Wazuh</strong>, where we’ll:</p>
<ul style="font-size:1em;">
  <li>Correlate events from <strong>Wireshark</strong>, <strong>pfSense</strong>, and <strong>Snort</strong></li>
  <li>Generate <strong>actionable alerts</strong></li>
  <li>Produce <strong>executive-level reports</strong></li>
</ul>


<h2 style="color:#2471A3; font-size:1.6em;">🔍 Phase 3: Wazuh – Security Event Monitoring & Response</h2>

<h3 style="color:#;">🎯 Objective</h3>
<p>Correlate logs and respond to security incidents by integrating <strong>pfSense</strong>, <strong>Snort</strong>, and <strong>endpoint agents</strong> with <strong>Wazuh</strong> for unified visibility and real-time response.</p>

<h3 style="color:#;">🧩 Key Actions</h3>
<ul style="font-size:1em;">
  <li>Configured log forwarding from <strong>pfSense</strong> and endpoints to the <strong>Wazuh manager</strong></li>
  <li>Detected <strong>privilege escalation</strong>, <strong>SSH brute-force</strong>, and <strong>suspicious user behavior</strong></li>
  <li>Correlated alerts with anomalies observed earlier in <strong>Wireshark</strong></li>
</ul>

<hr>

<h3 style="color:#;">📡 Configure pfSense to Forward Logs to Wazuh</h3>
<ol style="font-size:1em; padding-left:1.2em;">
  <li>Go to <code>Status → System Logs → Settings</code></li>
  <li>Enable <strong>Remote Logging</strong></li>
  <li>Set <strong>Wazuh Server IP:</strong> <code>192.168.1.xxx</code></li>
  <li>Select log types:
    <ul>
      <li>Firewall</li>
      <li>Snort alerts</li>
      <li>System logs</li>
    </ul>
  </li>
  <li>Protocol: <strong>UDP</strong></li>
  <li>Port: <strong>514</strong></li>
</ol>
<p style="font-style:italic; color:#;">🔗 This ensures that all pfSense events are sent to Wazuh for centralized monitoring.</p>

<hr>

<h3 style="color:#;">🛠️ Configure Wazuh Manager to Receive Logs</h3>
<p>On the <strong>Wazuh Manager</strong> server:</p>

<pre style="background-color:#eef; padding:1em; border-radius:6px;">
sudo systemctl status wazuh-manager
</pre>
<p>✅ Confirm that <code>rsyslog</code> is listening for incoming pfSense logs.</p>

<h4 style="color:#2471A3;">📥 Check for Active Data Flow</h4>
<pre style="background-color:#eef; padding:1em; border-radius:6px;">
sudo tail -f /var/ossec/logs/ossec.log
</pre>
<p>🔎 You should see log entries indicating receipt of pfSense and Snort data.</p>

<hr>

<h3 style="color:#;">🖥️ Confirm the Wazuh Agent is Active on Ubuntu</h3>
<p>Run the following command on the endpoint:</p>

<pre style="background-color:#eef; padding:1em; border-radius:6px;">
sudo systemctl status wazuh-agent
</pre>

<h4 style="color:#2471A3;">✅ Expected Output:</h4>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px;">
Active: active (running)
</pre>
<p>This confirms that the endpoint agent is properly reporting logs, user activity, and file integrity data to the Wazuh manager.</p>

<h2 style="color:#2471A3; font-size:1.6em;">🧪 Simulate Attacks Again</h2>

<p>To validate detection and response, repeat the following simulated attacks:</p>

<h3 style="color:#;">💥 Attack Commands</h3>
<pre style="background-color:#eef; padding:1em; border-radius:6px;">
# Nmap scan (stealth)
nmap -sS -p- 192.168.1.xxx

# Ping flood
ping -f 192.168.1.xxx

# SSH brute-force attack using Medusa
medusa -h 192.168.1.xxx -u root -P /usr/share/wordlists/rockyou.txt -M ssh
</pre>

<p>These tests generate real-time alerts in <strong>Wireshark</strong>, <strong>Snort</strong>, and <strong>Wazuh</strong> simultaneously.</p>

<hr>

<h3 style="color:#;">🕵️ Investigate in Wazuh Dashboard</h3>
<p>Access the Wazuh Web UI:</p>
<pre style="background-color:#eef; padding:1em; border-radius:6px;">
https://192.168.1.xxx
</pre>

<p>Login with your credentials, then navigate to:</p>
<pre style="background-color:#f4f4f4; padding:1em; border-radius:6px;">
Security Events → Event Viewer
</pre>

<h4 style="color:#2471A3;">🔎 Filter by:</h4>
<ul style="font-size:1em;">
  <li><strong>Agent:</strong> Ubuntu</li>
  <li><strong>Rule Level:</strong> ≥ 8</li>
  <li><strong>Tactic:</strong> Credential Access, Reconnaissance</li>
</ul>

<hr>

<h3 style="color:#;">📋 Review Alerts and Correlated Events</h3>
<p>Expected alerts include:</p>
<ul style="font-size:1em;">
  <li>🚨 <strong>SSH brute-force attempt</strong> (Medusa)</li>
  <li>🌊 <strong>ICMP flood</strong> (Ping flood)</li>
  <li>🔍 <strong>Nmap port scan</strong></li>
  <li>⚙️ <strong>Privilege escalation</strong> (sudo misuse)</li>
</ul>

<h4 style="color:#2471A3;">📌 Each alert includes:</h4>
<ul style="font-size:1em;">
  <li>Timestamp</li>
  <li>Source IP</li>
  <li>Rule ID</li>
  <li>Mapped MITRE ATT&CK tactic</li>
  <li>Severity Level</li>
</ul>

<p style="font-style:italic; color:#;">🧠 Wazuh correlates events across multiple sources, revealing attack patterns and their context within the overall intrusion timeline.</p>

<hr>

<h3 style="color:#;">📡 Snort Alert Integration with Wazuh</h3>
<p>Even when Snort’s blocking mode is disabled, it continues to detect and forward:</p>
<ul style="font-size:1em;">
  <li>Nmap scans</li>
  <li>ICMP floods</li>
  <li>Suspicious TCP flags</li>
</ul>

<h4 style="color:#2471A3;">🧠 These appear in Wazuh as:</h4>
<ul style="font-size:1em;">
  <li><strong>Snort:</strong> NMAP TCP Scan</li>
  <li><strong>Snort:</strong> ICMP Flood Detected</li>
</ul>

<p style="font-style:italic; color:#;">📡 Snort functions as a network sensor, feeding Wazuh actionable intelligence for deeper event correlation.</p>


<h2 style="color:#2471A3; font-size:1.6em;">📊 Findings and Analysis</h2>

<p>Wazuh correlated multiple events observed across the SOC environment:</p>
<ul style="font-size:1em;">
  <li>🔐 <strong>SSH brute-force attempts</strong> from Kali attacker</li>
  <li>🌊 <strong>ICMP floods</strong> targeting Ubuntu victim</li>
  <li>⚙️ <strong>Privilege escalation</strong> via sudo misuse</li>
</ul>

<h4 style="color:#2471A3;">🧠 Mapped MITRE ATT&CK Tactics:</h4>
<ul style="font-size:1em;">
  <li><strong>Credential Access</strong></li>
  <li><strong>Reconnaissance</strong></li>
  <li><strong>Privilege Escalation</strong></li>
</ul>

<p>These correlations confirmed earlier <strong>Wireshark</strong> and <strong>Snort</strong> findings and provided a unified, high-fidelity alert trail for investigation.</p>

<hr>

<h3 style="color:#;">📋 Summary of Phase 3</h3>
<table style="width:100%; border-collapse:collapse; font-size:0.95em; border:1px solid #ccc;">
  <thead style="background-color:#f0f0f0;">
    <tr>
      <th style="border:1px solid #ccc; padding:0.6em;">Tool</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Function</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Key Detection</th>
      <th style="border:1px solid #ccc; padding:0.6em;">Outcome</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">Wazuh Manager</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Central log correlation</td>
      <td style="border:1px solid #ccc; padding:0.6em;">SSH brute-force, privilege escalation</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Detected</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">pfSense Logs</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Firewall, Snort data</td>
      <td style="border:1px solid #ccc; padding:0.6em;">ICMP, TCP SYN floods</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Forwarded</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">Snort IDS</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Intrusion detection</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Nmap, ICMP alerts</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Integrated</td>
    </tr>
    <tr>
      <td style="border:1px solid #ccc; padding:0.6em;">Wireshark</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Traffic verification</td>
      <td style="border:1px solid #ccc; padding:0.6em;">Correlated packet captures</td>
      <td style="border:1px solid #ccc; padding:0.6em;">✅ Confirmed</td>
    </tr>
  </tbody>
</table>

<hr>

<h4 style="color:#2471A3;">🧩 Key Insight</h4>
<p style="font-size:1em;">
Wazuh acts as the <strong>brain of the SOC</strong> — combining firewall, IDS, and endpoint telemetry into a single, correlated security view that enables faster detection, investigation, and incident response.
</p>

<hr>

<h3 style="color:#2471A3;">✅ End of Phase 3 – Wazuh Integration Complete</h3>
<p style="font-size:1em;">
Next, we can proceed to the <strong>Conclusion & Recommendations</strong> section to summarize the SOC findings and propose improvements.
</p>




<h2 style="color:#2471A3; font-size:1.6em;">🧠 5. Final Findings & Impact</h2>

<p>The engagement confirmed that <strong>Femlalogy’s network</strong> was susceptible to multiple attack vectors targeting internal endpoints.</p>
<p>Simulated attacks—including <strong>Nmap scans</strong>, <strong>ping floods</strong>, and <strong>SSH brute-force login attempts</strong>—were successfully executed from the attacker machine (<strong>Kali</strong>) and captured across <strong>Wireshark</strong>, <strong>pfSense</strong>, and <strong>Wazuh</strong>.</p>

<h3 style="color:#;">🔍 Key Findings</h3>
<ul style="font-size:1em;">
  <li>🔓 <strong>Open SSH Port (22)</strong> on the victim machine was identified via Nmap, exposing it to brute-force attempts.</li>
  <li>🌊 <strong>ICMP traffic</strong> was initially unrestricted, enabling ping flood attacks until firewall rules were applied.</li>
  <li>🛡️ <strong>Snort alerts</strong> confirmed detection of reconnaissance and anomalous TCP patterns.</li>
  <li>📡 <strong>Wazuh SIEM</strong> correlated multiple indicators—privilege escalation, failed SSH logins, and abnormal user behavior—into unified, actionable alerts.</li>
  <li>🌍 <strong>GeoIP filtering</strong> and <strong>ransomware IP blocking</strong> in pfSense effectively reduced inbound threats from high-risk regions and known malicious hosts.</li>
</ul>

<p style="font-style:italic; color:#;">✅ These results validate the importance of layered defense, log correlation, and real-time visibility in protecting critical assets.</p>

<hr>

<h2 style="color:#2471A3; font-size:1.6em;">🛠️ 6. Recommendations</h2>
<p>To strengthen Femlalogy’s cybersecurity posture, the following measures are advised:</p>

<h4 style="color:#2471A3;">🔐 Enforce Least Privilege Access</h4>
<ul style="font-size:1em;">
  <li>Restrict SSH access to trusted IPs</li>
  <li>Use key-based authentication to mitigate brute-force risks</li>
</ul>

<h4 style="color:#2471A3;">📦 Maintain Updated Blocklists</h4>
<ul style="font-size:1em;">
  <li>Regularly update pfSense aliases for ransomware and threat IPs</li>
  <li>Integrate live threat intelligence feeds</li>
</ul>

<h4 style="color:#2471A3;">🚫 Enable Snort Blocking Mode</h4>
<ul style="font-size:1em;">
  <li>Activate inline blocking to automatically drop packets flagged as malicious</li>
</ul>

<h4 style="color:#2471A3;">🌍 Expand GeoIP Filtering</h4>
<ul style="font-size:1em;">
  <li>Continuously refine regional filtering rules</li>
  <li>Deny traffic from high-risk geographies</li>
</ul>

<h4 style="color:#2471A3;">📡 Implement Centralized Logging</h4>
<ul style="font-size:1em;">
  <li>Forward all endpoint, firewall, and IDS logs to Wazuh</li>
  <li>Enable consolidated analysis and long-term retention</li>
</ul>

<h4 style="color:#2471A3;">🧪 Conduct Regular Simulations</h4>
<ul style="font-size:1em;">
  <li>Schedule red team exercises</li>
  <li>Evaluate detection and response workflows under realistic attack conditions</li>
</ul>

<h4 style="color:#2471A3;">🎓 Educate Staff on Threat Awareness</h4>
<ul style="font-size:1em;">
  <li>Provide ongoing security awareness training</li>
  <li>Cover phishing, password hygiene, and incident reporting</li>
</ul>

<hr>

<h2 style="color:#2471A3; font-size:1.6em;">📘 7. Conclusion</h2>

<p>This SOC engagement demonstrated how <strong>Femlalogy</strong> can detect, analyze, and respond to evolving cyber threats using a fully open-source security stack.</p>

<ul style="font-size:1em;">
  <li>🔬 <strong>Wireshark</strong> enabled deep packet inspection and traffic analysis</li>
  <li>🛡️ <strong>pfSense</strong> enforced network segmentation, firewall rules, and intrusion detection policies</li>
  <li>📡 <strong>Wazuh</strong> served as the central SIEM, correlating alerts and mapping them to MITRE ATT&CK tactics</li>
</ul>

<p style="font-style:italic; color:#;">✅ The project highlights the effectiveness of a proactive, layered defense strategy and the critical role of continuous monitoring.</p>
<p style="font-size:1em;">With the proposed improvements, Femlalogy is now better equipped to strengthen its resilience, minimize exposure, and respond decisively to modern cybersecurity threats.</p>